In [1]:
# show result of last expression even if it was an assignment
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

# Step 1: The Station List

In [ ]:
import json
from pathlib import Path

# JSON data from 
# https://layer.bicyclesharing.net/map/v1/fgb/map-inventory
map_json_path = Path("./map.json")
with map_json_path.open() as f:
    map_json = json.load(f)
    
map_json

In [ ]:
from typing import NamedTuple

class Station(NamedTuple):
    lat: float
    lon: float
    name: str
    id: str
       
    @property
    def lonlat(self):
        return self.lon, self.lat
       
    @property
    def latlon(self):
        return self.lat, self.lon
        
    def __repr__(self):
        return f"{self.lat:9.4f} {self.lon:9.4f} {self.id:10} {self.name[:30]}"

In [ ]:
stations = [Station(
    lat=feature["geometry"]["coordinates"][1],
    lon=feature["geometry"]["coordinates"][0],
    name=feature["properties"]["name"],
    id=feature["properties"]["terminal"],
) for feature in map_json["features"] if feature["properties"]["installed"]]

In [ ]:
from collections import Counter
prefixes = Counter([st.id.split("-")[0] for st in stations])
prefixes.most_common()

In [ ]:
stations = [st for st in stations if st.id[:2] in {"OK", "BK", "EM"}]
len(stations)

# Step 2: Folium for Mapping

In [ ]:
import folium

def basemap(stations):
    center_lat = 0.5 * (min(st.lat for st in stations) + max(st.lat for st in stations))
    center_lon = 0.5 * (min(st.lon for st in stations) + max(st.lon for st in stations))
    
    mymap = folium.Map(
        location=[center_lat, center_lon], 
        zoom_start=12,
        tiles='Stamen Terrain',
    )

    for st in stations:
        folium.Circle(
            location=st.latlon, 
            radius=50,
            popup=f"<strong>{st.id}</strong> {st.name}",
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1.0,
        ).add_to(mymap)
        
    return mymap

basemap(stations)

# Step 3: Traveling Salesman with OR Tools

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

manager = pywrapcp.RoutingIndexManager(
    len(stations),  # number of stations
    1,              # number of vehicles
    0,              # index of start/finish station
)
routing = pywrapcp.RoutingModel(manager)

In [ ]:
from geopy.distance import distance

def as_the_bird_flies(from_station: Station, to_station: Station):
    """Returns distance between two stations in whole meters"""
    dist = distance(from_station.latlon, to_station.latlon)
    return int(dist.meters)  

def cost_function(from_index, to_index):
    """Cost function for OR Tools"""
    from_station = stations[manager.IndexToNode(from_index)]
    to_station = stations[manager.IndexToNode(to_index)]
    return as_the_bird_flies(from_station, to_station)

In [ ]:
# not a particularly Pythonic API...
callback_ptr = routing.RegisterTransitCallback(cost_function)
routing.SetArcCostEvaluatorOfAllVehicles(callback_ptr)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.time_limit.seconds = 120

# This lines will run for a few minutes!
assignment = routing.SolveWithParameters(search_parameters)

In [ ]:
total_dist = assignment.ObjectiveValue()
print(f"Distance: {total_dist/1000:.3} km ({total_dist/1609:.3} mi)")

In [ ]:
def route_from_ortools_solution(assignment, routing, manager):
    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route.append((
            stations[manager.IndexToNode(previous_index)],
            stations[manager.IndexToNode(index)]
        ))
    return route

route = route_from_ortools_solution(assignment, routing, manager)

In [ ]:
routemap = basemap(stations)

for st1, st2 in route:
    folium.PolyLine([st1.latlon, st2.latlon]).add_to(routemap)

routemap

# Step 4: Real cycling directions from Mapbox

In [ ]:
import mapbox

service = mapbox.Directions(access_token="pk.eyJ1Ijoiam9uZW1vIiwiYSI6ImNqaXJ6MWV1ZjFtcTUzdm1mMWZ0YXNsZ2oifQ.6zNZ3Eadouh2aFfEtbIdew", host="localhost:8001")

In [ ]:
class Route(NamedTuple):
    distance: float
    duration: float
    polyline: folium.PolyLine


def get_route_from_mapbox(from_station, to_station):
    """Fetch directions from Mapbox API and turn into Folium polyline"""
    resp = service.directions(
        [from_station.lonlat, to_station.lonlat],
        profile="mapbox/cycling",
        geometries="polyline",
    ).geojson()
    
    # flip coordinates from lon/lat to lat/lon
    coordinates = resp["features"][0]["geometry"]["coordinates"]
    flipped = [(lat, lon) for (lon, lat) in coordinates]
    
    return Route(
        distance=resp["features"][0]["properties"]["distance"],
        duration=resp["features"][0]["properties"]["duration"],
        polyline=folium.PolyLine(flipped),
    )

In [ ]:
import itertools

frommapbox = {}
for from_station, to_station in itertools.combinations(stations, 2):
    
    # If the straight line distance is less than one and a half
    # miles, then get the cycling distance from the Mapbox API.
    straight_line = as_the_bird_flies(from_station, to_station)
    if straight_line < 1.5 * 1609:
        
        # Both directions because the route might be different
        # due to one-way streets, etc.
        frommapbox[(from_station, to_station)] = \
            get_route_from_mapbox(from_station, to_station)
        frommapbox[(to_station, from_station)] = \
            get_route_from_mapbox(to_station, from_station)

In [ ]:
from random import sample
debugmap = basemap(stations)

for st1, st2 in sample(frommapbox.keys(), 500):
    folium.PolyLine([st1.latlon, st2.latlon], weight=1).add_to(debugmap)
    
debugmap

In [ ]:
debugmap2 = basemap(stations)
frommapbox[(stations[4], stations[10])].polyline.add_to(debugmap2)
debugmap2

# Step 5: Traveling Salesman, take 2

In [ ]:
def cost_function_2(from_index, to_index):
    """Cost function for OR Tools"""
    from_station = stations[manager.IndexToNode(from_index)]
    to_station = stations[manager.IndexToNode(to_index)]
    
    try:
        route = frommapbox[(from_station, to_station)]
        return int(route.duration)  # in seconds
    
    except KeyError:
        # overestimate duration for routes we don't
        # have directions for (really want to avoid them)
        return as_the_bird_flies(from_station, to_station) * 5

In [ ]:
manager_2 = pywrapcp.RoutingIndexManager(len(stations), 1, 0)
routing_2 = pywrapcp.RoutingModel(manager_2)
callback_ptr_2 = routing_2.RegisterTransitCallback(cost_function_2)
routing_2.SetArcCostEvaluatorOfAllVehicles(callback_ptr_2)
search_parameters_2 = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_2.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
assignment_2 = routing_2.SolveWithParameters(search_parameters_2)

In [ ]:
total_dist = assignment_2.ObjectiveValue()
print(f"Duration: {total_dist/3600:.3} hrs")

In [ ]:
route_2 = route_from_ortools_solution(assignment_2, routing_2, manager_2)

In [ ]:
routemap_2 = basemap(stations)

for st1, st2 in route_2:
    try:
        polyline = frommapbox[(st1, st2)].polyline
        polyline.add_to(routemap_2)
    except KeyError:
        folium.PolyLine([st1.latlon, st2.latlon], color="pink").add_to(routemap_2)

routemap_2

In [ ]:
def cost_function_3(from_index, to_index):
    """Cost function for OR Tools"""
    from_station = stations[manager.IndexToNode(from_index)]
    to_station = stations[manager.IndexToNode(to_index)]
    
    try:
        route = frommapbox[(from_station, to_station)]
        return int(abs(120 - route.duration))  # seconds
    
    except KeyError:
        # grossly overestimate duration for routes we don't
        # have directions for (really want to avoid them)
        return as_the_bird_flies(from_station, to_station) * 5

In [ ]:
manager_3 = pywrapcp.RoutingIndexManager(len(stations), 1, 0)
routing_3 = pywrapcp.RoutingModel(manager_3)
callback_ptr_3 = routing_3.RegisterTransitCallback(cost_function_3)
routing_3.SetArcCostEvaluatorOfAllVehicles(callback_ptr_3)
search_parameters_3 = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_3.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
assignment_3 = routing_3.SolveWithParameters(search_parameters_3)

In [ ]:
route_3 = route_from_ortools_solution(assignment_3, routing_3, manager_3)

In [ ]:
routemap_3 = basemap(stations)

for st1, st2 in route_3:
    polyline = frommapbox[(st1, st2)].polyline
    polyline.add_to(routemap_3)

routemap_3

In [ ]:
routemap_3.save("mapexport.html")